<a href="https://colab.research.google.com/github/swapnalihinukale/swapnalihinukale/blob/main/Audio_Text_sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keyBERT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.8 MB/s eta 0:00:00
  Created wheel for keyBERT: filename=keybert-0.7.0-py3-none-any.whl size=23800 sha256=085ed670a50a732b0f9dab411f363de0e7a935b50ab9af7f4894dfbe8aa5097c
  Stored in directory: /root/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install nrclex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nrclex: filename=NRCLex-3.0.0-py3-none-any.whl size=43329 sha256=de49cf60ec18111e91bdb8bef5b4d7440fb8899b385975eb216256e6efddff3a
  Stored in directory: /root/.cache/pip/wheels/83/95/c0/42b43fb15eb48e4f5a67cba8915540cb2783591c59c037a9e5
Successfully built nrclex


In [ ]:
!pip install pydub
!pip install pip install SpeechRecognition

In [ ]:
# Import necessary libraries
from pydub import AudioSegment
import speech_recognition as sr
import os

import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import email
from nltk.corpus import stopwords
nltk.download('punkt')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
pd.options.mode.chained_assignment = None


import os
import glob
from textblob import TextBlob

import nltk.data


#nltk.download('punkt')
from nltk.collocations import *
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
nltk.download('stop_words')
nltk.download('wordnet')


from nltk.corpus import stopwords, wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

from keybert import KeyBERT

from nrclex import NRCLex


# **Spliting Audio file into chunks (20s)**

In [ ]:
# Input audio file to be sliced
audio = AudioSegment.from_wav("/content/drive/MyDrive/audio_text_sentiment/audio.wav")
n = len(audio)

# Variable to count the number of sliced chunks
counter = 1

# Interval length at which to slice the audio file.
# If length is 86 seconds, and interval is 20 seconds,
# The chunks created will be:
# chunk1 : 0 - 20 seconds
# chunk2 : 20 - 40 seconds
# chunk3 : 40 - 60 seconds

interval = 20 * 1000
time=[]

overlap = 0.0 * 1000

# Initialize start and end seconds to 0
start = 0
end = 0

# Flag to keep track of end of file.
# When audio reaches its end, flag is set to 1 and we break
flag = 0

# Iterate from 0 to end of the file,
# with increment = interval
for i in range(0, n, interval):
    # During first iteration,
    # start is 0, end is the interval
    if i == 0:
        start = 0
        end = interval
    
    # All other iterations,
    # start is the previous end - overlap
    # end becomes end + interval
    else:
        start = end - overlap
        end = start + interval
        
    # When end becomes greater than the file length,
    # end is set to the file length
    # flag is set to 1 to indicate break.
    if end >= n:
        end = n
        flag = 1
        
    # Storing audio file from the defined start to end 
    chunk = audio[start:end]
    # Filename / Path to store the sliced audio
    
    filename = '/content/drive/MyDrive/audio_text_sentiment/chunks/chunk'+str(counter)+'.wav'
    
    # Store the sliced audio file to the defined path
    chunk.export(filename, format ="wav")
    # Print information about the current chunk
    print("Processing chunk "+str(counter)+". Start = "+str(start)+" end = "+str(end))
    
    a = str(start/1000)+ "-"+str(end/1000)
    time.append(a)
    # Increment counter for the next chunk
    counter = counter + 1
    # Slicing of the audio file is done.
    # Skip the below steps if there is some other usage
    # for the sliced audio files.
    

AUDIO_FILE = filename

# Initialize the recognizer
r = sr.Recognizer()

# Traverse the audio file and listen to the audio
with sr.AudioFile(AUDIO_FILE) as source:
    audio_listened = r.listen(source)

# Try to recognize the listened audio
# And catch expections.
try:
    rec = r.recognize_google(audio_listened)
    
# If recognized, write into the file.
# fh.write(rec+" ")
# If google could not understand the audio
except sr.UnknownValueError:
    print("Could not understand audio")
    
# If the results cannot be requested from Google.
# Probably an internet connection error.
except sr.RequestError as e:
    print("Could not request results.")
    
# Check for flag.
# If flag is 1, end of the whole audio reached.
# Close the file and break.
# if flag == 1:
#     break



# **Audio To Text Function**


In [ ]:
def audio_to_text(filename):
    '''
        Converts speech to text
    '''
    # Get recognizer
    r = sr.Recognizer() 
    
    with sr.AudioFile(filename) as source:
        audio_listened = r.record(source) 

        # Try to recognize the listened audio 
        # And catch expections. 
        try:     
            return r.recognize_google(audio_listened, show_all = False) 
            

        # If google could not understand the audio 
        except sr.UnknownValueError: 
            print("Could not understand audio provided") 
            return None

        # If the results cannot be requested from Google. 
        # Probably an internet connection error. 
        except sr.RequestError as e: 
            print("Could not request results.") 
            return None

# **Chunks to Text Conversion**

In [ ]:
def chunkAudioToText(): 
    '''
    gives text output of chunks
    '''
    
    i = 1
    answer=[]
    for chunk in os.listdir('/content/drive/MyDrive/audio_text_sentiment/chunks'): 
        file = '/content/drive/MyDrive/audio_text_sentiment/chunks/chunk'+str(i)+'.wav'
        print("Processing chunk "+str(i)) 
        rec = audio_to_text(file) # function which  do Speech to text conversion(IBM Watson SpeechToText API)
        answer.append(rec)
        if rec == "Error5487":
            return "Error5487E"
        
        i += 1
    
    
    return answer

# **Creating DataFrame**

In [ ]:
d1 = {"Timestamp": time, 'Answer':chunkAudioToText()}
df = pd.DataFrame.from_dict(d1)
df

In [ ]:
df.to_csv("/content/drive/MyDrive/Audio_Data.csv", index=False)

# **cleaned Text**


In [4]:
 def readExcelFile(input_file_path):
    df = pd.read_csv(input_file_path)
    return df

   
 def clean_column(data):
    if data is not None:

        data=str(data) # convert to string data type
        data = data.lower()#lower casing
        #replcae with space
        data = re.sub('-', '', data)
        data = re.sub('_', '', data)
        # Remove data between square brackets
        data =re.sub('\[[^]]*\]', '', data)
        # removes punctuation
        data = re.sub(r'[^\w\s]','',data)
        data = re.sub(r'\n',' ',data)
        data = re.sub(r'[0-9]+','',data)
        data = re.sub(r"[.?]+",'. ' ,data) 
        # removal of urls
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        data=url_pattern.sub(r'',data)
        #removal html tags
        html_pattern = re.compile('<.*?>')
        data = html_pattern.sub('', data)
        data = re.sub(r'\n','' ,data) #remove newline
        data = re.sub(r'\r','' ,data)
        data = re.sub(r'/c', '' ,data)
        data = re.sub(r"[^\w'.,?\s]+",' ' ,data) #remove all except words, req punctuation marks and space
        data = data.encode('utf-8').decode('ascii','ignore') #remove non-ascii characters
        data = re.sub(r'\s+',' ' ,data) #remove extra space b/w words
        #chat word conversion
        data = re.sub(r"\'ve", " have ", data)
        data = re.sub(r"can't", "cannot ", data)
        data = re.sub(r"n't", " not ", data)
        data = re.sub(r"I'm", "I am", data)
        data = re.sub(r" m ", " am ", data)
        data = re.sub(r"\'re", " are ", data)
        data = re.sub(r"\'d", " would ", data)
        data = re.sub(r"\'ll", " will ", data)
        
        return data
        
        
def CleanedReviewTextT():
  df=readExcelFile('/content/drive/MyDrive/Audio_Data.csv') # csv file
 

  #split the whole review on punctuation(.) in single sentense
  s = df["Answer"].str.split('.',expand = True).apply(pd.Series,1).stack()
  s.index = s.index.droplevel(-1)
  s.name = 'Answer'
  x = s.to_frame(name='new_review')
  df = df.join(x)
  #apply clean column
  df['new_review'] = df['new_review'].apply(clean_column)
  #remove stopwords
  stop_words = set(stopwords.words("english"))
  df['Answer']=str(df['Answer'])

  df['CleanedReviewText'] = df['new_review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
  

  return df    

# **Sentiments**

In [ ]:
def sentiment_scores(sentence): 
  
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = TextBlob(sentence).sentiment[0]
  
    return sid_obj

def Sentiment_column():
  df=CleanedReviewTextT()
  
  df.drop(df.filter(regex="Unname"),axis=1, inplace=True)

  df['CleanedReviewText'] = df['CleanedReviewText'].astype(str)
  df['Sentiment_score_subsent'] = df['CleanedReviewText'].apply(sentiment_scores)
  df['Sentiment_per_subsentence'] = ["positive" if x>0 else("Negative" if x<0 else "Neutral") for x in df["Sentiment_score_subsent"] ]
  df['ReviewLength'] = df['CleanedReviewText'].apply(len)
  df['sentiment_substance_review_length'] = df['Sentiment_score_subsent']*df['ReviewLength']
  df['polarity_sign'] = np.sign(df.Sentiment_score_subsent)
  df.drop('Answer',axis=1,inplace=True)
  return df

In [ ]:
Sentiment_column()

# **Keyword_list**

In [ ]:
def Keyword_list():
  df=Sentiment_column()
  df= df.explode('CleanedReviewText').reset_index()
  df['keywords_list'] = pd.Series()
  df['keywords_score'] = pd.Series()


  kw_model = KeyBERT(model='all-mpnet-base-v2')
  for i in range(0,len(df.CleanedReviewText)):
      
      doc = str(df['CleanedReviewText'][i])
    
      stop_words = "english"

      keyword = kw_model.extract_keywords(doc,

            keyphrase_ngram_range=(0,1),

            stop_words='english',

            highlight=False,

            top_n=3)

     
      df['keywords_list'][i]= np.array(list(dict(keyword).keys()),dtype=object)
      df['keywords_score'][i]=np.array(list(dict(keyword).values()),dtype=object)

  return df

     
    

In [ ]:
Keyword_list()

# **Bigram**

In [ ]:
def bigram():
  df =Keyword_list()
  df['bigram_list'] = pd.Series()
  df['bigram_score'] = pd.Series()


  kw_model = KeyBERT(model='all-mpnet-base-v2')
  for i in range(len(df.CleanedReviewText)):
      # print(i)
      doc = str(df['CleanedReviewText'][i])
      # print(doc)
      stop_words = "english"

      keyword = kw_model.extract_keywords(doc,

            keyphrase_ngram_range=(0,2),

            stop_words='english',

            highlight=False,

            top_n=3)

     
      df['bigram_list'][i]= np.array(list(dict(keyword).keys()),dtype=object)
      df['bigram_score'][i]=np.array(list(dict(keyword).values()),dtype=object)

  return df    

# **Trigram**

In [ ]:
def trigram():
  df =bigram() 
  df['trigram_list'] = pd.Series()
  df['trigram_score'] = pd.Series()


  kw_model = KeyBERT(model='all-mpnet-base-v2')
  for i in range(len(df.CleanedReviewText)):
      # print(i)
      doc = str(df['CleanedReviewText'][i])
      # print(doc)
      stop_words = "english"

      keyword = kw_model.extract_keywords(doc,

            keyphrase_ngram_range=(0,3),

            stop_words='english',

            highlight=False,

            top_n=3)

     
      df['trigram_list'][i]= np.array(list(dict(keyword).keys()),dtype=object)
      df['trigram_score'][i]=np.array(list(dict(keyword).values()),dtype=object)

  return df   

In [ ]:

def emotion(x):
    text = NRCLex(x)
    if text.top_emotions[0][1] == 0.0:
        return "No emotion"
    else:
        return text.top_emotions[0][0]
    
def emotion_column():
  df=trigram()
  df['Emotion'] = df['CleanedReviewText'].apply(emotion)
  return df

# **EDA**

In [ ]:
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
df=Sentiment_column()
sns.countplot(df['Sentiment_per_subsentence'])

In [ ]:
df=emotion_column()
df.drop('index',inplace=True,axis=1)
df


In [ ]:
df.to_csv("/content/drive/MyDrive/audio_text_sentiment/audio_output.csv")